In [1]:
import re
from langchain_community.vectorstores import FAISS
from langchain.docstore.document import Document
import bs4
import ssl
import urllib3
import pandas as pd
import faiss
import os
from langchain.embeddings import HuggingFaceEmbeddings
import torch
import gc

def instruct_structure(prompt):
    input_text, output_text = prompt.split('### target')
    input_text = input_text.replace('### glossaries', '### glossary').replace('\n* ', '\n• ')
    input_text = re.sub(r"\[[^\]]+\] ", "[UNK] ", input_text)
    return input_text

project_id = "prod-ai-project"

from google.cloud import bigquery
client = bigquery.Client(project=project_id)
sql = """select series_id, episode_id, org_input_text, org_output_text, prompt 
        from webtoon_translation.structured_240820_ep_line
        where data_split = 'romance_valid'"""
df = client.query(sql).result().to_dataframe()
from tqdm import tqdm
tqdm.pandas()
df['prompt'] = df['prompt'].progress_apply(lambda x: instruct_structure(x))

/home/bun.2/.cache/pypoetry/virtualenvs/poetry-env-eDEwtiIl-py3.10/lib/python3.10/site-packages/google/cloud/bigquery/table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(
100%|████████████████████████████████████████████████████████████████████████████████████████████████| 74/74 [00:00<00:00, 18823.37it/s]


In [2]:
data_idx = 1
data = df['prompt'][data_idx]
example = data.split("### source")[1].strip()

In [3]:
print(example)

000	[UNK] 키에에엑-!!
001	[UNK] 후우.
002	[UNK] 놀랍군.
003	[UNK] 꺅?!
004	[UNK] 마치 신이 내리는 천벌 같구나.
005	[UNK] 이런 솜씨의 마법사는 처음이다.
006	[UNK] 넌 사람의 아이가 맞느냐?
007	[UNK] 당연하죠.
008	[UNK] 굳이 말하자면 미래의 최종 보스랍니다.
009	[UNK] 그렇겠지.
010	[UNK] 몇 발짝만 나가면 추락하겠어.
011	[UNK] 설마 이 정도로는 부족하다고 여기서 던지려는 건 아니지?!
